In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from sklearn import svm, cross_validation
import pylab as pl
from PIL import Image
import numpy as np
import os

/usr/lib/pymodules/python2.7/matplotlib/__init__.py:1173: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
# Make sure that caffe is on the python path:
caffe_root = '/u/zexuan/caffe/caffe/'  # this file is expected to be in {caffe_root}/examples
caffe_real_root = '/pkgs/caffe/'
thesis_root = '/ais/gobi2/pingpong/thesis/'
#!ls /pkgs/caffe
import sys
sys.path.insert(0, caffe_real_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

if not os.path.isfile('~/caffe/caffe/models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print("Downloading pre-trained CaffeNet model...")
    !~/caffe/caffe/scripts/download_model_binary.py ~/caffe/caffe/models/bvlc_reference_caffenet

Model already exists.


In [3]:
caffe.set_device(0)
caffe.set_mode_gpu()

In [4]:
# MODEL_FILE = caffe_root +'models/bvlc_reference_caffenet/deploy.prototxt'
# PRETRAINED = caffe_root +'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
# caffe.set_mode_cpu()
# net = caffe.Classifier(MODEL_FILE, PRETRAINED,
#                        mean=np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1),
#                        channel_swap=(2,1,0),
#                        raw_scale=255,
#                        image_dims=(600, 600))
# print net.blobs['data'].data.shape
# [(k, v.data.shape) for k, v in net.blobs.items()]


# caffe.set_mode_cpu()
print caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
#!ls caffe_root+'models/bvlc_reference_caffenet/deploy.prototxt'
net = caffe.Net(caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
                caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB
print net.blobs['data'].data.shape
[(k, v.data.shape) for k, v in net.blobs.items()]

/u/zexuan/caffe/caffe/models/bvlc_reference_caffenet/deploy.prototxt
(10, 3, 227, 227)


[('data', (10, 3, 227, 227)),
 ('conv1', (10, 96, 55, 55)),
 ('pool1', (10, 96, 27, 27)),
 ('norm1', (10, 96, 27, 27)),
 ('conv2', (10, 256, 27, 27)),
 ('pool2', (10, 256, 13, 13)),
 ('norm2', (10, 256, 13, 13)),
 ('conv3', (10, 384, 13, 13)),
 ('conv4', (10, 384, 13, 13)),
 ('conv5', (10, 256, 13, 13)),
 ('pool5', (10, 256, 6, 6)),
 ('fc6', (10, 4096)),
 ('fc7', (10, 4096)),
 ('fc8', (10, 1000)),
 ('prob', (10, 1000))]

In [5]:
import skimage
import skimage.io
import time

start_time = time.time()
training_set = {}
training_set['target'] = []
container_path = 'labeled_img_selected_winning_classes/'

folders = [f for f in sorted(os.listdir(container_path)) if os.path.isdir(os.path.join(container_path, f))]
print 'folders: ', folders

result = []
for folder in folders:
    folder_path = os.path.join(container_path, folder)
    documents = [os.path.join(folder_path, d) for d in sorted(os.listdir(folder_path))]
    input_10_imgs = []
    for pic in documents:
        if not pic.endswith('png'):
            continue
        img = skimage.img_as_float(skimage.io.imread(pic)).astype(np.float32)
        input_10_imgs.append(img)
        if len(input_10_imgs) >= 10:
            #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
            #net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
            net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), input_10_imgs)
            out = net.forward()
            result.append(np.mean(net.blobs['fc6'].data, axis=0))
            input_10_imgs = []
        training_set['target'].append(folder)
#training_set['data']=data
#training_set['target']=target
print 'Total time loading training data: ', time.time()-start_time, ' seconds'
print 'number of images: ', len(training_set['target'])
#print 'dmensionality of each image: ', input_10_imgs[0].shape

folders:  ['C21_TOP_PLAYER_UNDER_NET_selected', 'C22_TOP_PLAYER_HIT_OUT_selected', 'C23_TOP_PLAYER_FOREHAND_MISS_HIT_selected', 'C24_TOP_PLAYER_BACKHAND_MISS_HIT_selected', 'C25_BOTTOM_PLAYER_UNDER_NET_selected', 'C26_BOTTOM_PLAYER_HIT_OUT_selected', 'C27_BOTTOM_PLAYER_FOREHAND_MISS_HIT_selected', 'C28_BOTTOM_PLAYER_BACKHAND_MISS_HIT_selected']
Total time loading training data:  41.7512168884  seconds
number of images:  1010


In [6]:
start_time = time.time()
testing_set = {}
testing_set['target'] = []
container_path = 'labeled_img_selected_winning_classes_testset/'

folders = [f for f in sorted(os.listdir(container_path)) if os.path.isdir(os.path.join(container_path, f))]
print 'folders: ', folders

for folder in folders:
    folder_path = os.path.join(container_path, folder)
    documents = [os.path.join(folder_path, d) for d in sorted(os.listdir(folder_path))]
    input_10_imgs = []
    for pic in documents:
        if not pic.endswith('png'):
            continue
        img = skimage.img_as_float(skimage.io.imread(pic)).astype(np.float32)
        input_10_imgs.append(img)
        if len(input_10_imgs) >= 10:
            #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
            #net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
            net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), input_10_imgs)
            out = net.forward()
            result.append(np.mean(net.blobs['fc6'].data, axis=0))
            input_10_imgs = []
        testing_set['target'].append(folder)
#training_set['data']=data
#training_set['target']=target
print 'Total time loading training data: ', time.time()-start_time, ' seconds'
print 'number of images: ', len(testing_set['target'])
#print 'dmensionality of each image: ', input_10_imgs[0].shape

folders:  ['C21_TOP_PLAYER_UNDER_NET_selected', 'C22_TOP_PLAYER_HIT_OUT_selected', 'C23_TOP_PLAYER_FOREHAND_MISS_HIT_selected', 'C24_TOP_PLAYER_BACKHAND_MISS_HIT_selected', 'C25_BOTTOM_PLAYER_UNDER_NET_selected', 'C26_BOTTOM_PLAYER_HIT_OUT_selected', 'C27_BOTTOM_PLAYER_FOREHAND_MISS_HIT_selected', 'C28_BOTTOM_PLAYER_BACKHAND_MISS_HIT_selected']
Total time loading training data:  8.52292490005  seconds
number of images:  230


In [7]:
# print caffe.io.load_image(thesis_root + 'labeled_image_selected/bottom_player_winning_selected/point_00001_frame_00033.png').shape
# print caffe.io.load_image(thesis_root + 'labeled_image_selected/bottom_player_winning_selected/point_00001_frame_00033.png')[0][0]
# print training_set['data'][0][0][0]

'''
result = []
#for i in xrange(2200, 2300,10):
for i in xrange(0, traning_set_size, 10):
    #net.blobs['data'].data[...] = transformer.preprocess('data', training_set['data'][0])
    net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data', x), training_set['data'][i:i+10])
    out = net.forward()
    #print net.blobs['fc7'].data.shape
    result.append(np.mean(net.blobs['fc6'].data, axis=0))
'''

print 'Number of input to SVM: ', len(result), 'Size of each input', result[0].shape

print transformer.deprocess('data', net.blobs['data'].data[0]).shape
print 'Sample image from the training set'
#plt.imshow(transformer.deprocess('data', net.blobs['data'].data[4]))

Number of input to SVM:  124 Size of each input (4096,)
(227, 227, 3)
Sample image from the training set


In [8]:
import pickle

#pickle.dump(result, open("nn_result_fc6_unnormalized_clf_7videos.pickle", "wb" ) )

In [9]:
#from sklearn.preprocessing import normalize
print len(result)
print np.max(result[0])
print result[0].shape
normalized_result = []
#normalized_result = normalize(result)
for r in result:
    #normalized_result.append(r/np.max(r))
    normalized_result.append((r-np.mean(r))/np.std(r))

124
33.6298
(4096,)


In [10]:
#pickle.dump(normalized_result, open("nn_result_fc6_normalized_clf_7videos.pickle", "wb" ) )

In [11]:
print len(normalized_result)
print np.max(normalized_result[0])
print normalized_result[0].shape
print result[0].shape


124
10.2349
(4096,)
(4096,)


In [12]:
from sklearn import decomposition

traning_set_size = len(training_set['target'])
testing_set_size = len(testing_set['target'])
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(result, training_set['target'][0:traning_set_size:10], test_size=0.25, random_state=42)
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(normalized_result, training_set['target'][0:traning_set_size:10], test_size=0.25, random_state=2)

X_train = normalized_result[:traning_set_size/10]
X_test = normalized_result[traning_set_size/10:]
y_train = training_set['target'][0:traning_set_size:10]
y_test = testing_set['target'][0:traning_set_size:10]

print 'X_train: %d; X_test: %d; y_train: %d; y_test: %d'%(len(X_train), len(X_test), len(y_train), len(y_test))

X_train: 101; X_test: 23; y_train: 101; y_test: 23


In [13]:
for c in [0.00001, 0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 100, 1000]:
    svc = svm.SVC(kernel='linear', C=c) 
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

C = 1e-05, Cross validation: [ 0.15384615  0.2         0.28        0.2       ]
C = 1e-05, Mean value: 0.208462
C = 0.001, Cross validation: [ 0.38461538  0.36        0.2         0.4       ]
C = 0.001, Mean value: 0.336154
C = 0.005, Cross validation: [ 0.5   0.64  0.44  0.68]
C = 0.005, Mean value: 0.565
C = 0.01, Cross validation: [ 0.5   0.64  0.6   0.52]
C = 0.01, Mean value: 0.565
C = 0.05, Cross validation: [ 0.53846154  0.4         0.44        0.52      ]
C = 0.05, Mean value: 0.474615
C = 0.1, Cross validation: [ 0.46153846  0.48        0.56        0.6       ]
C = 0.1, Mean value: 0.525385
C = 1, Cross validation: [ 0.73076923  0.64        0.52        0.56      ]
C = 1, Mean value: 0.612692
C = 10, Cross validation: [ 0.65384615  0.36        0.6         0.6       ]
C = 10, Mean value: 0.553462
C = 100, Cross validation: [ 0.42307692  0.48        0.48        0.44      ]
C = 100, Mean value: 0.455769
C = 1000, Cross validation: [ 0.53846154  0.56        0.64        0.32      ]
C =

In [14]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=1)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

C = 0.01, Cross validation: [ 0.23076923  0.2         0.2         0.2       ]
C = 0.01, Mean value: 0.207692
C = 0.05, Cross validation: [ 0.19230769  0.08        0.24        0.12      ]
C = 0.05, Mean value: 0.158077
C = 0.1, Cross validation: [ 0.34615385  0.16        0.2         0.16      ]
C = 0.1, Mean value: 0.216538
C = 1, Cross validation: [ 0.26923077  0.2         0.16        0.08      ]
C = 1, Mean value: 0.177308
C = 10, Cross validation: [ 0.42307692  0.64        0.44        0.4       ]
C = 10, Mean value: 0.475769
C = 100, Cross validation: [ 0.65384615  0.44        0.48        0.76      ]
C = 100, Mean value: 0.583462
C = 1000, Cross validation: [ 0.53846154  0.4         0.44        0.44      ]
C = 1000, Mean value: 0.454615
C = 1e+06, Cross validation: [ 0.53846154  0.64        0.6         0.36      ]
C = 1e+06, Mean value: 0.534615


In [15]:
for c in [0.01, 0.05, 0.1, 1, 10, 100, 1000, 1000000]:
    svc = svm.SVC(C=c, kernel='poly', degree=2)
    kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
    cv_result = cross_validation.cross_val_score(svc, X_train, y_train, cv=kfold, n_jobs=1)
    print 'C = %g, Cross validation: %s'%(c, cv_result)
    print 'C = %g, Mean value: %g'%(c, np.mean(cv_result))

C = 0.01, Cross validation: [ 0.15384615  0.2         0.2         0.16      ]
C = 0.01, Mean value: 0.178462
C = 0.05, Cross validation: [ 0.19230769  0.12        0.08        0.16      ]
C = 0.05, Mean value: 0.138077
C = 0.1, Cross validation: [ 0.15384615  0.24        0.12        0.24      ]
C = 0.1, Mean value: 0.188462
C = 1, Cross validation: [ 0.07692308  0.2         0.04        0.16      ]
C = 1, Mean value: 0.119231
C = 10, Cross validation: [ 0.5   0.56  0.52  0.52]
C = 10, Mean value: 0.525
C = 100, Cross validation: [ 0.57692308  0.48        0.56        0.56      ]
C = 100, Mean value: 0.544231
C = 1000, Cross validation: [ 0.46153846  0.16        0.6         0.48      ]
C = 1000, Mean value: 0.425385
C = 1e+06, Cross validation: [ 0.53846154  0.6         0.52        0.68      ]
C = 1e+06, Mean value: 0.584615


In [16]:
from sklearn import svm, cross_validation
X_train = np.asarray(X_train)
svc = svm.SVC(kernel='precomputed')
kernel_train = np.dot(X_train, X_train.T)  # linear kernel
svc.fit(kernel_train, y_train)

# Testing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
kernel_test = np.dot(X_test, X_train.T)
y_pred = svc.predict(kernel_test)
print accuracy_score(y_test, y_pred)

import pandas as pd
y_true = pd.Series(y_test)
y_pred = pd.Series(y_pred)

#print pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

print confusion_matrix(y_test, y_pred).shape
print confusion_matrix(y_test, y_pred)
#print '\n'.join(sorted(list(set(y_test))))
print len(set(y_test).union(set(y_pred)))

0.0869565217391
(8, 8)
[[1 0 1 0 0 3 0 0]
 [1 0 0 0 1 0 0 0]
 [0 1 0 1 1 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 2 0 0 3 1 0]
 [0 0 0 0 0 1 1 0]
 [0 0 1 0 0 1 0 0]
 [0 0 0 0 1 0 1 0]]
8


/u/zexuan/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [23]:
clf_best = svm.SVC(C=1, kernel='linear').fit(X_train, y_train)
clf_best.score(X_test, y_test)

0.086956521739130432

In [19]:
print len(X_test)
print len(y_test)
for i,_ in enumerate(X_test):
    if clf_best.predict(X_test[i])[0] == y_test[i]:
        print 'SAME: ', clf_best.predict(X_test[i])[0]
    else:
        print 'DIFFERENT: ', clf_best.predict(X_test[i])[0], y_test[i]
        

23
23
SAME:  C21_TOP_PLAYER_UNDER_NET_selected
DIFFERENT:  C26_BOTTOM_PLAYER_HIT_OUT_selected C21_TOP_PLAYER_UNDER_NET_selected
DIFFERENT:  C23_TOP_PLAYER_FOREHAND_MISS_HIT_selected C21_TOP_PLAYER_UNDER_NET_selected
DIFFERENT:  C26_BOTTOM_PLAYER_HIT_OUT_selected C21_TOP_PLAYER_UNDER_NET_selected
DIFFERENT:  C26_BOTTOM_PLAYER_HIT_OUT_selected C21_TOP_PLAYER_UNDER_NET_selected
DIFFERENT:  C25_BOTTOM_PLAYER_UNDER_NET_selected C22_TOP_PLAYER_HIT_OUT_selected
DIFFERENT:  C21_TOP_PLAYER_UNDER_NET_selected C22_TOP_PLAYER_HIT_OUT_selected
DIFFERENT:  C24_TOP_PLAYER_BACKHAND_MISS_HIT_selected C23_TOP_PLAYER_FOREHAND_MISS_HIT_selected
DIFFERENT:  C22_TOP_PLAYER_HIT_OUT_selected C23_TOP_PLAYER_FOREHAND_MISS_HIT_selected
DIFFERENT:  C25_BOTTOM_PLAYER_UNDER_NET_selected C23_TOP_PLAYER_FOREHAND_MISS_HIT_selected
DIFFERENT:  C22_TOP_PLAYER_HIT_OUT_selected C24_TOP_PLAYER_BACKHAND_MISS_HIT_selected
DIFFERENT:  C26_BOTTOM_PLAYER_HIT_OUT_selected C25_BOTTOM_PLAYER_UNDER_NET_selected
DIFFERENT:  C26_BOTT

/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationW

In [36]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

kfold = cross_validation.KFold(len(X_train), n_folds=4, shuffle=True)
cv_result = cross_validation.cross_val_score(neigh, X_train, y_train, cv=kfold, n_jobs=1)

print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)
    
clf = neigh.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

Cross validataion result [ 0.26923077  0.2         0.36        0.2       ] . Avg: 0.257307692308
Testing result 0.130434782609


In [21]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
cv_result = cross_validation.cross_val_score(gnb, X_train, y_train, cv=kfold, n_jobs=1)
print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)

clf = gnb.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

Cross validataion result [ 0.42307692  0.36        0.4         0.4       ] . Avg: 0.395769230769
Testing result 0.130434782609


In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cv_result = cross_validation.cross_val_score(lr, X_train, y_train, cv=kfold, n_jobs=1)
print 'Cross validataion result', cv_result, '. Avg:', np.mean(cv_result)

clf = lr.fit(X_train, y_train)
print 'Testing result', clf.score(X_test, y_test)

Cross validataion result [ 0.42307692  0.36        0.4         0.72      ] . Avg: 0.475769230769
Testing result 0.0869565217391


In [24]:
#predict_probability = clf.predict_proba(X_test)
#print predict_probability[0]
#print sum(predict_probability[0]

print X_test[0]
print clf.predict(X_test[0])

[-0.04419233  7.54528666 -0.295926   ..., -0.295926   -0.295926   -0.295926  ]
['C21_TOP_PLAYER_UNDER_NET_selected']


/u/zexuan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [ ]:
pickle.dump(clf, open("fc6_normalized_lr_clf_7videos_winning_cls.pickle", "wb" ) )